In [ ]:
# pip install pytrends pandas matplotlib seaborn
# если запускаешь в colab: !pip install pytrends

from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

# ---------- Настройки ----------
queries = [
    "зарплата junior разработчик",
    "зарплата it",
    "зарплата python разработчик",
    "зарплата it-специалиста"
]

# timeframe: 5 лет до текущей даты. Измени, если нужно.
timeframe = "2025-01-01 2025-09-19"  # или "today 5-y"
# гео: RU и два региона
geos = {
    "Russia": "RU",
    "Moscow": "RU-MOW",
    "SaintPetersburg": "RU-SPE"
}

out_dir = "google_trends_output"
os.makedirs(out_dir, exist_ok=True)

# Параметры подключения к Google Trends
pytrends = TrendReq(hl='ru-RU', tz=180)  # tz Moscow UTC+3 -> 180 minutes

# helper: разбить на батчи по 5 (исправленная версия)
def chunks(lst, n=5):
    """Разбивает список на части по n элементов"""
    if not lst:  # если список пустой
        return []
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# 1) Загрузка interest_over_time для каждой фразы и гео
for geo_name, geo_code in geos.items():
    query_batches = list(chunks(queries, 5))  # преобразуем в список
    if not query_batches:  # проверка на пустые батчи
        print(f"[WARN] Нет запросов для {geo_name}")
        continue
        
    for batch in query_batches:
        try:
            print(f"Обрабатываю батч для {geo_name}: {batch}")
            pytrends.build_payload(batch, timeframe=timeframe, geo=geo_code)
            iot = pytrends.interest_over_time()
            
            if iot.empty:
                print(f"[WARN] пустой результат для {geo_name} / {batch}")
            else:
                # Создаем безопасное имя файла
                safe_batch_names = [q.replace(' ', '_').replace('/', '_') for q in batch]
                fname = f"{out_dir}/iot_{geo_name}_{'_'.join(safe_batch_names)}.csv"
                iot.to_csv(fname, index=True)
                print(f"[OK] Saved interest over time: {fname}")
            
            time.sleep(1.5)
            
        except Exception as e:
            print(f"[ERROR] При загрузке interest_over_time для {geo_name} / {batch}: {e}")
            time.sleep(5)

# 2) Загрузка interest_by_region
for q in queries:
    try:
        print(f"Загружаем данные по регионам для: {q}")
        pytrends.build_payload([q], timeframe=timeframe, geo="RU")
        by_region = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)
        
        if not by_region.empty:
            safe_q = q.replace(' ', '_').replace('/', '_')
            fname = f"{out_dir}/by_region_{safe_q}.csv"
            by_region.to_csv(fname)
            print(f"[OK] Saved interest by region for '{q}': {fname}")
        else:
            print(f"[WARN] No region data for '{q}'")
            
        time.sleep(1.5)
        
    except Exception as e:
        print(f"[ERROR] При загрузке interest_by_region для '{q}': {e}")
        time.sleep(5)

# 3) Строим графики по interest_over_time
sns.set(style="whitegrid", context="talk")
for geo_name, geo_code in geos.items():
    files = [f for f in os.listdir(out_dir) if f.startswith(f"iot_{geo_name}_") and f.endswith(".csv")]
    
    if not files:
        print(f"[WARN] Нет файлов для {geo_name}")
        continue
        
    for f in files:
        try:
            df = pd.read_csv(os.path.join(out_dir, f), index_col=0, parse_dates=True)
            if 'isPartial' in df.columns:
                df = df.drop(columns=['isPartial'])
            
            plt.figure(figsize=(12,6))
            for col in df.columns:
                plt.plot(df.index, df[col], label=col)
            
            plt.title(f"Google Trends — interest over time ({geo_name})\n{f.replace('.csv','')}")
            plt.ylabel("Relative interest (0-100)")
            plt.xlabel("Date")
            plt.legend()
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            png = os.path.join(out_dir, f.replace('.csv', '.png'))
            plt.savefig(png, dpi=150)
            plt.close()
            print(f"[OK] Saved plot {png}")
            
        except Exception as e:
            print(f"[ERROR] При обработке файла {f}: {e}")

# 4) Для каждого запроса построим карту/бар по топ-10 регионов
for q in queries:
    safe_q = q.replace(' ', '_').replace('/', '_')
    fname = f"{out_dir}/by_region_{safe_q}.csv"
    
    if os.path.exists(fname):
        try:
            br = pd.read_csv(fname, index_col=0)
            if q in br.columns:
                br_sorted = br.sort_values(by=q, ascending=False).head(10)
                
                plt.figure(figsize=(10,6))
                sns.barplot(x=br_sorted[q].values, y=br_sorted.index)
                plt.title(f"Top-10 регионов по интересу к '{q}' (Россия)")
                plt.xlabel("Interest (0-100)")
                plt.ylabel("Region")
                plt.tight_layout()
                
                png = fname.replace('.csv', '.png')
                plt.savefig(png, dpi=150)
                plt.close()
                print(f"[OK] Saved region bar {png}")
            else:
                print(f"[WARN] Колонка '{q}' не найдена в файле {fname}")
                
        except Exception as e:
            print(f"[ERROR] При обработке файла {fname}: {e}")
    else:
        print(f"[WARN] Region file not found for '{q}': {fname}")

print("Готово. Результаты в папке:", out_dir)

In [4]:
from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt

# Настройка
pytrends = TrendReq(hl='ru-RU', tz=180)

# Ключевые запросы для анализа
base_keyword = "зарплата it"
related_queries = [
    "зарплата программист",
    "зарплата разработчик",
    "зарплата айтишник", 
    "зарплата it специалист",
    "зарплата в it",
    "it зарплаты",
    "зарплата junior разработчик",
    "зарплата middle разработчик",
    "зарплата senior разработчик"
]

# Анализ связанных запросов
pytrends.build_payload([base_keyword], timeframe='today 12-m', geo='RU')
related = pytrends.related_queries()
top_related = related[base_keyword]['top']

print("Топ связанных запросов:")
print(top_related.head(10))

Топ связанных запросов:
                     query  value
0  зарплата it специалиста    100
